In [65]:
# import sys
#!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz
# !pip install spacy
# !python3 -m spacy download en
# !pip install -U textblob
# !python3 -m textblob.download_corpora

[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [52]:
import pandas as pd
import numpy as np 
import requests, json
import pandas as pd, numpy as np
from api import ocr_key
from collections import OrderedDict
from spacy.matcher import PhraseMatcher
import spacy
import en_core_web_sm
from textblob import TextBlob

nlp = spacy.load('en')


In [24]:
ks = pd.read_excel('food.xlsx')

# ks.head(10)
mst_common = []
shrt = ks['Long_Desc'].tolist()

# for item in shrt:
#     item = item.split(',')
#     for i in item: 
#         mst_common[i] = mst_common.get(i, 0) + 1
    
# sort_items = sorted(mst_common.items(), key=lambda x: x[1], reverse=True)
3
# print(len(mst_common))

# sort_items

for item in shrt : 
    item = item.split(',')
    mst_common.append(item[0].lower())
    

mst_common = set(mst_common)
print("Unique enteries present in the data :" ,len(mst_common))


7793
Unique enteries present in the data : 918


In [7]:
# def FoodDatabase():
#     df = pd.read_excel("food.xlsx", usecols="B:C")
#     food_group_map = {
#         100: "dairy",
#         1800: "grain",
#         2000: "grain",
#         500: "meat",
#         700: "meat",
#         1000: "meat",
#         1300: "meat",
#         1500: "meat",
#         1700: "meat",
#         900: "fruit_veg",
#         1100: "fruit_veg",
#         1600: "fruit_veg",
#     }
#     df["Food Group"] = df["FdGrp_Cd"].map(food_group_map)
#     df_split = df["Long_Desc"].str.split(",", n=1, expand=True)
#     df_split.columns = ["Food Name", "Food Name Detail"]
#     df = pd.concat([df, df_split], axis=1)
#     df = df.groupby("Food Group")["Food Name"].unique()
#     dairy = df["dairy"]
#     grain = df["grain"]
#     meat = df["meat"]
#     fruit_veg = df["fruit_veg"]
#     print(df)
    
#     return dairy, grain, meat, fruit_veg

In [8]:
# df = pd.read_excel("food.xlsx", usecols="B:C")
# print(df.head())

# food_group_map = {
#     100: "dairy",
#     1800: "grain",
#     2000: "grain",
#     500: "meat",
#     700: "meat",
#     1000: "meat",
#     1300: "meat",
#     1500: "meat",
#     1700: "meat",
#     900: "fruit_veg",
#     1100: "fruit_veg",
#     1600: "fruit_veg",
# }

# df["Food Group"] = df["FdGrp_Cd"].map(food_group_map)

# df_split = df["Long_Desc"].str.split(",", n=1, expand=True)
# df_split.columns = ["Food Name", "Food Name Detail"]


# df = pd.concat([df, df_split], axis=1)

# display(df)
# df = df.groupby("Food Group")["Food Name"].unique()
# dairy = df["dairy"]
# grain = df["grain"]
# meat = df["meat"]
# fruit_veg = df["fruit_veg"]


# print(df)



#### OCR SPACE IMPORTING 

In [2]:
def ocr_space_file(
    filename, overlay=False, api_key=ocr_key, language="eng", istable=True, scale=True
):
    """ OCR.space API request with local file.
        Python3.5 - not tested on 2.7

    :param filename: Your file path & name.
    :param overlay: Is OCR.space overlay required in your response.
                    Defaults to False.
    :param api_key: OCR.space API key.
                    Defaults to 'helloworld'.
    :param language: Language code to be used in OCR.
                    List of available language codes can be found on https://ocr.space/OCRAPI
                    Defaults to 'en'.
    :return: Result in JSON format.
    :param istable: parses output as a table
    """

    payload = {
        "isOverlayRequired": overlay,
        "apikey": api_key,
        "language": language,
        "istable": istable,
    }
    with open(filename, "rb") as f:
        r = requests.post(
            "https://api.ocr.space/parse/image", files={filename: f}, data=payload,
        )
    return r.content.decode()
  
def ocr_space_url(
    url, overlay=False, api_key=ocr_key, language="eng", isTable=True, scale=True
):
    """ OCR.space API request with remote file.
        Python3.5 - not tested on 2.7

    :param url: Image url.
    :param overlay: Is OCR.space overlay required in your response.
                    Defaults to False.
    :param api_key: OCR.space API key.
                    Defaults to 'helloworld'.
    :param language: Language code to be used in OCR.
                    List of available language codes can be found on https://ocr.space/OCRAPI
                    Defaults to 'en'.
    :return: Result in JSON format.
    :param isTable: parses output as a table
    """

    payload = {
        "url": url,
        "isOverlayRequired": overlay,
        "apikey": api_key,
        "language": language,
        "isTable": isTable,
    }
    r = requests.post("https://api.ocr.space/parse/image", data=payload,)
    # return r.content.decode()

    # Change this return in source code to return JSON object, not string
    return r.json()


# API response and obtaing "LineText"
def parse():
    data = ocr_space_url("https://ocr.space/Content/Images/receipt-ocr-original.jpg")
    lines = data.get("ParsedResults")[0]["TextOverlay"]["Lines"]
    words = [line.get("LineText").lower() for line in lines]
    return words

if __name__ == "__main__":
    parse()
    FoodDatabase()

Food Group
dairy        [Butter, Butter oil, Cheese, Cheese food, Chee...
fruit_veg    [Acerola, Acerola juice, Apples, Apple juice, ...
grain        [Bagels, Biscuits, Bread, Cake, Cookies, Puff ...
meat         [Chicken, Canada Goose, Duck, Goose, Guinea he...
Name: Food Name, dtype: object


### Building the FoodDB 

In [53]:
# Query the excel sheet and obtain the target food groups. Note not food group codes used
def food_database():
    df = pd.read_excel("food.xlsx", usecols="B:C")
    food_group_map = {
        100: "dairy",
        1800: "grain",
        2000: "grain",
        500: "meat",
        700: "meat",
        1000: "meat",
        1300: "meat",
        1500: "meat",
        1700: "meat",
        900: "fruit_veg",
        1100: "fruit_veg",
        1600: "fruit_veg",
    }
    df["Food Group"] = df["FdGrp_Cd"].map(food_group_map)
    df_split = df["Long_Desc"].str.split(",", n=1, expand=True)
    df_split.columns = ["Food Name", "Food Name Detail"]
    df = pd.concat([df, df_split], axis=1)
    df = df.groupby("Food Group")["Food Name"].unique()
    dairy = df["dairy"]
    grain = df["grain"]
    meat = df["meat"]
    fruit_veg = df["fruit_veg"]
    food_groups = np.concatenate((dairy, meat, grain, fruit_veg))
    # print(type(food_groups))
    #return dairy, grain, meat, fruit_veg
    return food_groups


In [54]:
def float_or_int(string):
    if string.isdigit():
        return True
    elif string.replace('.','',1).isdigit() and string.count('.') < 2:
        return True
    else:
        return False 

In [86]:
def filter_words(all_words):
    text_doc = nlp(all_words)
    
    # removing stop words or punctuation 
    text_doc2 = [] 
    for token in text_doc : 
        if(not token.is_stop and len(token.text) > 2 and not float_or_int(token.text)): 
#             print("token length == ",len(token), " token text len ==", len(token.text), " token text -", token.text)
            text_doc2.append(token.lemma_)
  
    words_string = ' '.join(text_doc2)
    
    text_blob_object = TextBlob(words_string)
    
    singular_string = ' '.join(text_blob_object.words.singularize())
    
#     plural_string = ' '.join(text_blob_object.words.pluralize())
    
    return(singular_string,words_string)
    

In [98]:
def patter_match():

    
    nlp = spacy.load('en')
    matcher = PhraseMatcher(nlp.vocab, attr = 'LOWER')
    
    parsed_words = parse()
'''
    parsed_words = ['walmart', 'save money. live better.', 
    '( 330 ) 339 -', '3991', 'manager diana earnest',
     '231 bluebell dr sw', 'new philadelphia oh 44663',
      '02115 009044 44', '01301', 'pet toy', 'floppy puppy', 
      'sssupreme s', 'z . 5 squeak', 'munchy dmbel', 'dog treat', 
      'ped pch 1', 'ped pch 1', 'coupon 23100', '1--inymd smores',
       'french drsng', '3 oranges', 'baby carrots', 'collards',
        'calzone', 'mm rvw mnt', 'stkobrlplabl', 'stkobrlplabl', 
        'stko sunflwr', 'stko sunflwr', 'stko sunflwr',
        'stko sunflwr', 'bling beads', 'great value', 
        'lipton', 'dry dog', 'tax', 'us debit', '004747571658', 
        '004747514846', '070060332153', '084699803238', '068113108796',
         '007119013654', '002310011802', '002310011802', '052310037000', '088491226837', '004132100655', '001466835001', '003338366602', '1', '000000004614k1', '005208362080 f', '003399105848', '001558679414', '001558679414', '001558679410', '001558679410', '001558679410', '001558679410', '076594060699', '007874203191 f', '001200011224 f', '002310011035', '1', 'subtotal', '6.750', 'total', 'visa tend', '5', '12 .', '9166', '1.97', '1.97', '4.97', '5.92', '3.77', '2.92', '0.50', '0.50', '1.00-0', '3.98', '1.98', '.47', '1.48', '1.24', '2.50', '19.77', '1.97', '1.97', '0.97', '0.97', '0.97', '0.97', '0.97', '9.97', '4.48', '44', '93.62', '4.59', '98.21', '98.21', '1', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'o', 'o', 'o', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'o', 'x', 'x', 'o', 'approval # 572868', 'ref # 720900544961', 'trans id', '387209239650894', 'validation - 87hs', 'payment service', 'aid a0000000980840', 'tc 51319ca81dcz2bc7', 'terminal # sc010764', '*signature ver ified', '07/28/17', '02 : 39 : 48', 'change due', '# items sold 25', '0443 0223 1059 8001 5140', 'low'
    , 'ices you can trust .', 'every', '0.00', 'day .', '07/28/17', 
    '02 : 39 : 48']
''' 
    
    words_string = ' '.join(parsed_words)
    
    (single,plural) = filter_words(words_string)
    
    
    text_doc_singular = nlp(single)
    text_doc_plural   = nlp(plural)


    food_groups = food_database()

    # Building the intial list of keywords we want to match the parsed items aganist
    patterns = [nlp(text) for text in food_groups]
    matcher.add("Food Matcher", None, *patterns)    
    matches = matcher(text_doc_singular)
    
    results = []
    # printing all th matches, TerminologyList is the name of the our patter matcher
    for i in range(len(matches)):    
        match_id, start, end = matches [0]
#         results.append((nlp.vocab.strings[match_id], text_doc_singular[start:end]))
        results.append(text_doc_singular[start:end])

    matches = matcher(text_doc_plural)
    

    for i in range(len(matches)):    
        match_id, start, end = matches [0]
#         results.append((nlp.vocab.strings[match_id],text_doc_plural[start:end]))
        results.append(text_doc_plural[start:end])
    
    results = dict((i, results.count(i)) for i in results)
    
    return results

In [99]:
patter_match()

************
walmart save money live well manager diana earnest bluebell new philadelphium pet toy floppy puppy sssupreme squeak munchy dmbel dog treat pe pch ped pch coupon inymd smore french drsng orange baby carrot collard calzone rvw mnt stkobrlplabl stkobrlplabl stko sunflwr stko sunflwr stko sunflwr stko sunflwr bling bead great value lipton dry dog tax debit 000000004614k1 subtotal total visa tend approval ref tran validation 87h payment service aid a0000000980840 51319ca81dcz2bc7 terminal sc010764 signature ver ifie 07/28/17 change item sell low ice trust day 07/28/17
************
walmart save money live well manager diana earnest bluebell new philadelphia pet toy floppy puppy sssupreme squeak munchy dmbel dog treat pe pch ped pch coupon -inymd smore french drsng oranges baby carrots collards calzone rvw mnt stkobrlplabl stkobrlplabl stko sunflwr stko sunflwr stko sunflwr stko sunflwr bling beads great value lipton dry dog tax debit 000000004614k1 subtotal total visa tend appro